In [2]:
import ee
import datetime

# Initialize the Earth Engine library
ee.Initialize()

polygon = ee.FeatureCollection("users/sakdahomhuan/udfire_site");
geometry = polygon.geometry().bounds()

# Function to reproject and clip the image
def prepare_data(image):
    return image.reproject(crs='EPSG:3857', scale=500).clip(geometry)

# Function to convert Kelvin to Celsius
def kelvin_to_celsius(image):
    return image.multiply(0.02).subtract(273.15).set('system:time_start', image.get('system:time_start'))

# Function to load and process the LST data
def load_data(date_range):
    lst_collection = ee.ImageCollection('MODIS/061/MOD11A2') \
        .filterDate(date_range[0], date_range[1]) \
        .select('LST_Day_1km') \
        .map(prepare_data) \
        .map(kelvin_to_celsius)
        
    mean_lst_celsius = lst_collection.mean()
    return mean_lst_celsius




In [9]:
# Define the date range for the analysis
start_date = '2024-01-01'
end_date = '2024-05-30'
date_range = (start_date, end_date)

# Load the data
mean_lst_celsius = load_data(date_range)

# Function to export the image to Google Drive
def export_image_to_drive(image, description, folder, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region,
        crs='EPSG:3857'
    )
    task.start()
    

# Export the mean LST image to Google Drive
export_image_to_drive(mean_lst_celsius, 'Mean_LST_Celsius_2024', '_GEE_TUM', 500, geometry)
print(f'Exporting to Google Drive successfully')


Exporting to Google Drive successfully
